In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
pip install mlflow

In [2]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [3]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch.optim as optim
from torchinfo import summary


In [4]:
#utils

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn.functional as F
from tqdm.autonotebook import tqdm
import time
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from torchinfo import summary
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from typing import DefaultDict, Any, Callable, Optional


def run_epoch(
    model,
    optimizer,
    data_loader,
    loss_func,
    device,
    results: DefaultDict[str, list],
    score_funcs: Optional[dict[str, Callable[[np.ndarray, np.ndarray], float]]] = None,
    prefix: str = "",
    desc: Optional[str] =None,
    disable_tqdm: bool =False,
):
    running_loss = []
    y_true = []
    y_pred = []
    start = time.time()
    for inputs, labels in tqdm(data_loader, desc=desc, leave=False, disable=disable_tqdm):
        inputs = inputs.to(device)
        labels = labels.to(device)

        y_hat = model(inputs)
        loss = loss_func(y_hat, labels)

        if model.training:
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        running_loss.append(loss.item())

        if score_funcs is not None:
            labels = labels.detach().cpu().numpy()
            y_hat = y_hat.detach().cpu().numpy()
            y_true.extend(labels)
            y_pred.extend(y_hat)

    end = time.time()
    results[prefix + " " + "loss"].append(np.mean(running_loss))

    y_pred = np.asarray(y_pred)
    y_true = np.asanyarray(y_true)

    if score_funcs is not None and len(score_funcs) > 0:
        for score_name , score_func in score_funcs.items():
            results[prefix + " " + score_name].append(score_func(y_pred, y_true))
            mlflow.log_metric(prefix + " " + score_name, results[prefix + " " + score_name][-1], step=len(results["epoch"]))
    return end - start

def train_network(
    model,
    loss_func,
    train_loader,
    valid_loader=None,
    test_loader=None,
    epochs=10,
    device='cpu',
    score_funcs=None,
    checkpoint_file_save: Optional[str] =None,
    checkpoint_file_load: Optional[dict[str, Any]] =None,
    lr_schedule=None,
    optimizer=None,
    disable_tqdm=False,
    checkpoint_every_x: Optional[int]=None,
):
    model.to(device)
    if optimizer is None:
        optimizer = torch.optim.AdamW(model.parameters())

    if checkpoint_file_load:
        print('loading model ....')
        checkpoint = torch.load(checkpoint_file_load)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        results = checkpoint['results']
        total_train_time = checkpoint['results']['total time'][-1]
        mlflow.log_params({
            'resume_from_checkpoint':checkpoint_file_load,
            'start_epoch': start_epoch})
    else:
        results = defaultdict(list)
        start_epoch = 0
        total_train_time = 0

    total_train_time = 0

    for epoch in tqdm(range(start_epoch, epochs), desc='Epoch', disable=disable_tqdm):
        model.train()
        results['epoch'].append(epoch)
        total_train_time += run_epoch(
            model=model,
            optimizer=optimizer,
            data_loader=train_loader,
            loss_func=loss_func,
            device=device,
            results=results,
            score_funcs=score_funcs,
            prefix='train',
            desc='training'
        )
        results['total time'].append(total_train_time)
        mlflow.log_metric("train_loss", results['train loss'][-1],step=epoch)
        if valid_loader is not None:
            model.eval()
            with torch.no_grad():
                run_epoch(
                    model=model,
                    optimizer=optimizer,
                    data_loader=valid_loader,
                    loss_func=loss_func,
                    device=device,
                    results=results,
                    score_funcs=score_funcs,
                    prefix="valid",
                    desc='validating',
                )
            mlflow.log_metric("valid_loss", results['valid loss'][-1],step=epoch)

        if test_loader is not None:
            model.eval()
            with torch.no_grad():
                run_epoch(
                    model=model,
                    optimizer=optimizer,
                    data_loader=test_loader,
                    loss_func=loss_func,
                    device=device,
                    results=results,
                    score_funcs=score_funcs,
                    prefix="test",
                    desc='testing',
                )
            mlflow.log_metric("test_loss", results['test loss'][-1],step=epoch)

        if lr_schedule:
            if isinstance(lr_schedule, torch.optim.lr_scheduler.ReduceLROnPlateau):
                lr_schedule.step(results['valid loss'][-1])
            else:
                lr_schedule.step()
        
        if checkpoint_every_x and (epoch+1) % checkpoint_every_x == 0:
            file_name = f'{checkpoint_file_save.split(".")[0]}_{epoch+1}.pth'
            torch.save(
                {
                'results': results,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
                }, file_name)
            mlflow.log_artifact(file_name)
    if checkpoint_file_save:
        torch.save(
            {
                'results': results,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            },
            checkpoint_file_save
        )
        mlflow.log_artifact(checkpoint_file_save)
    return pd.DataFrame.from_dict(results)


def plot_data_and_predictions(x, y, y_pred, close=True):
    sns.scatterplot(x=x, y=y, color='blue', label='Data')
    sns.lineplot(x=x, y=y_pred.ravel(), color='red', label='Model')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Data and Model')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig


def plot_loss(loss, close=True):
    sns.lineplot(loss, label='loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.title('Loss')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig


def roc_auc_score_micro_wrapper(y_pred, y_true):
    if np.isnan(y_pred).any():
        print('Found NaN in predictions')
    if np.isnan(y_true).any():
        print('Found nan in y_true')
    if y_pred.ndim == 2:
        return roc_auc_score(y_true, y_pred[:, 1], average='micro')
    else:
        return roc_auc_score(y_true, y_pred, average='micro', multi_class='ovr')

def accuracy_score_wrapper(y_pred, y_true, threshold=0.5):
    if y_pred.ndim >= 2:
        pred = np.argmax(y_pred, axis=1)
    else:
        y_pred = (y_pred >= threshold).astype(int)
    return accuracy_score(pred, y_true)

def f1_score_wrapper(y_pred, y_true, threshold=0.5):
    if y_pred.ndim >= 2:
        pred = np.argmax(y_pred, axis=1)
    else:
        y_pred = (y_pred >= threshold).astype(int)
    return f1_score(pred, y_true)


def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)

def weight_reset(pytorch_model):
    if "reset_parameters" in dir(pytorch_model):
        pytorch_model.reset_parameters()
    return

<ipython-input-4-79c40c02a4a5>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
#models


import torch.nn as nn


def model_linear(D, hidden_layer_neurons, classes):
    model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(D, hidden_layer_neurons),
    nn.Tanh(),
    nn.Linear(hidden_layer_neurons, classes),
    )
    return model

def model_cnn(D, C, filters, K, classes):
    model = nn.Sequential(
    nn.Conv2d(C, filters, K, padding=K//2),
    nn.Tanh(),
    nn.Flatten(),
    nn.Linear(filters*D, classes),
    )
    return model

def model_cnn_pool(D, C, filters, K, classes):
    model = nn.Sequential(
    nn.Conv2d(C, filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(filters, filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(filters, filters, K, padding=K//2),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Conv2d(filters, 2*filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(2*filters, 2*filters, K, padding=K//2),
    nn.Tanh(),
    nn.Conv2d(2*filters, 2*filters, K, padding=K//2),
    nn.Tanh(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(2*filters*D//4**2, classes),
    )
    return model



## 1

In [7]:
mlflow.set_experiment('Exercise_1')

2025/04/09 08:42:38 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/674194671375701503', creation_time=1744188158264, experiment_id='674194671375701503', last_update_time=1744188158264, lifecycle_stage='active', name='Exercise_1', tags={}>

In [8]:
mnist_data_train = torchvision.datasets.MNIST(
    "./data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    )
mnist_data_test = torchvision.datasets.MNIST(
    "./data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
    )

batch_size = 32
mnist_train_loader = DataLoader(
    mnist_data_train,
    batch_size=batch_size,
    shuffle=True
    )
mnist_test_loader = DataLoader(
    mnist_data_test,
    batch_size=batch_size
    )

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.62MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 128kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.40MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.79MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
train_transforms = transforms.Compose(
    [
        transforms.RandomAffine(
            degrees=5,
            translate=(0.05, 0.05),
            scale=(0.98, 1.02),
            ),
        transforms.ToTensor(),
    ]
)
test_transforms = transforms.ToTensor()
mnist_train_augmented = torchvision.datasets.MNIST(
    './data_augmented',
    train=True,
    download=True,
    transform=train_transforms,
)
mnist_test_augmented = torchvision.datasets.MNIST(
    './data_augmented',
    train=False,
    download=True,
    transform=test_transforms,
)

mnist_train_augmented_loader = DataLoader(
    mnist_train_augmented,
    shuffle=True,
    batch_size=batch_size,
    num_workers=4
    )
mnist_test_augmented_loader = DataLoader(
    mnist_test_augmented,
    batch_size=batch_size,
    num_workers=4
    )

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.63MB/s]


Extracting ./data_augmented/MNIST/raw/train-images-idx3-ubyte.gz to ./data_augmented/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 148kB/s]


Extracting ./data_augmented/MNIST/raw/train-labels-idx1-ubyte.gz to ./data_augmented/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.40MB/s]


Extracting ./data_augmented/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data_augmented/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.96MB/s]

Extracting ./data_augmented/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data_augmented/MNIST/raw



In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 40
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}
def plot_results(data_df, close=True):
    sns.lineplot(data_df, x='epoch', y='valid Acc', label='valid Acc')
    plt.xlabel('epoch')
    plt.ylabel('valid Acc')
    plt.title('valid Acc')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

In [11]:

models = (model_linear(D=28*28, hidden_layer_neurons=256, classes=10),
          model_cnn(D=28*28, C=1, filters=16, K=3, classes=10),
          model_cnn_pool(D=28*28, C=1, filters=16, K=3, classes=10),
          model_cnn_pool(D=28*28, C=1, filters=16, K=3, classes=10))
train_loaders = (mnist_train_loader, mnist_train_loader, mnist_train_loader, mnist_train_augmented_loader)
test_loaders = (mnist_test_loader, mnist_test_loader, mnist_test_loader, mnist_test_augmented_loader)
name_experiment = (
    'linear_mnist',
    'cnn_mnist',
    'cnn_pool_mnist',
    'cnn_pool_augmented_mnist'
)

In [12]:
for i in range(4):
    print(i)
    model = models[i]
    train_loader = train_loaders[i]
    test_loader = test_loaders[i]
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    params['optimizer'] = optimizer.defaults
    
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))
    with mlflow.start_run(nested=True, run_name=name_experiment[i]):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

0


Epoch:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/1875 [00:00<?, ?it/s]

validating:   0%|          | 0/313 [00:00<?, ?it/s]

training:   0%|          | 0/1875 [00:00<?, ?it/s]

...

validating:   0%|          | 0/313 [00:00<?, ?it/s]

training:   0%|          | 0/1875 [00:00<?, ?it/s]

validating:   0%|          | 0/313 [00:00<?, ?it/s]

training:   0%|          | 0/1875 [00:00<?, ?it/s]

validating:   0%|          | 0/313 [00:00<?, ?it/s]

In [ ]:
!zip -r mlflow_data.zip /kaggle/working/mlruns

In [17]:
!ls /kaggle/working/


data  data_augmented  mlflow_data.zip  mlruns  model_summary.txt


## 2

In [5]:
mlflow.set_experiment("Exercise_2")

2025/04/13 12:05:02 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_2' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/106601540120253128', creation_time=1744545902640, experiment_id='106601540120253128', last_update_time=1744545902640, lifecycle_stage='active', name='Exercise_2', tags={}>

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
cifar_train = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transform,
    train=True,)
cifar_test = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transform,
    train=False,)

100%|██████████| 170M/170M [00:04<00:00, 34.1MB/s] 


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


In [7]:
batch_size = 256
cifar_train_loader = DataLoader(
    cifar_train,
    shuffle=True,
    batch_size=batch_size,
)

cifar_test_loader = DataLoader(
    cifar_test,
    batch_size=batch_size,
    )

In [6]:
# C = cifar_train[0][0].shape[0]
C = 3
filter = 16
K = 3
# w = cifar_train[0][0].shape[1]
# h = cifar_train[0][0].shape[2]
w , h = 32, 32
# classes = cifar_train[0][1].shape[0]
classes = 10
def build_model(num_conv_layers,
                num_pool_layers,
                num_hidden_layer=2, 
                init_hidden_size=512, 
                decay_factor=2,
                activation=nn.ReLU()):
    layers =[]
    in_channels = C
    out_channels = 32
    if num_pool_layers:
        pool_interval = max(1, num_conv_layers // (num_pool_layers + 1))
    else:
        pool_interval = num_conv_layers + 1
    
    currnet_pool_rounds = 0
    for i in range(num_conv_layers):
        layers.append(nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels,
            kernel_size=3,
            padding=3//2))
        layers.append(activation)
        in_channels = out_channels
        if (i+1) % pool_interval == 0 and currnet_pool_rounds < num_pool_layers:
            layers.append(nn.MaxPool2d(kernel_size=2))
            currnet_pool_rounds += 1
            out_channels *= 2
    final_w = w // (2 ** num_pool_layers)
    final_h = h // (2 ** num_pool_layers)
    fc_layers = []
    # Compute the number of features after flattening.
    in_features = in_channels * final_w * final_h

    fc_layers.append(nn.Flatten())

    if num_hidden_layer == 0:
        # Directly classify without extra hidden layers.
        fc_layers.append(nn.Linear(in_features, classes))
    else:
        # First FC layer: from flattened output to initial hidden size.
        fc_layers.append(nn.Linear(in_features, init_hidden_size))
        fc_layers.append(nn.ReLU(inplace=True))
        
        # Set the current hidden size that will be reduced in subsequent layers.
        current_hidden_size = init_hidden_size

        # Add additional hidden layers with decreasing size.
        for layer in range(1, num_hidden_layer):
            # Compute new hidden size with decay.
            new_hidden_size = max(10, current_hidden_size // decay_factor)
            fc_layers.append(nn.Linear(current_hidden_size, new_hidden_size))
            fc_layers.append(nn.ReLU(inplace=True))
            current_hidden_size = new_hidden_size

        # Final classification layer from the last hidden dimension to the number of classes.
        fc_layers.append(nn.Linear(current_hidden_size, classes))

    classifier = nn.Sequential(*fc_layers)
    model = nn.Sequential(*layers, classifier)
    return model

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs=20
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

### without optuna

In [10]:
for num_conv in range(2, 11):
    for num_pool in range(0,3):  
        print(f'num_conv:{num_conv}...num_pool:{num_pool}')
        model = build_model(
            num_conv_layers=num_conv,
            num_pool_layers=num_pool,
            num_hidden_layer=2, 
            init_hidden_size=512, 
            decay_factor=2,
            activation=nn.ReLU()
            )
        # optimizer = optim.SGD(model.parameters(), lr=0.001)
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
        params['optimizer'] = optimizer.defaults
        params['num_conv'] = num_conv
        params['num_pool'] = num_pool
        with open('model_summary.txt', 'w') as f:
            f.write(str(summary(model, input_size=(batch_size, 3, 32, 32))))
        with mlflow.start_run(nested=True, run_name=f'model{num_conv}_{num_pool}'):
            mlflow.log_params(params)
            mlflow.log_artifact('model_summary.txt')
            results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=cifar_train_loader,
            valid_loader=cifar_test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )
    

num_conv:2...num_pool:0


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

...

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
!zip -r mlflow_exercise_2.zip ./mlruns

### with optuna

In [ ]:
import optuna
from mlflow.types import Schema, TensorSpec
from mlflow.models import ModelSignature

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
activation_functions = {
'ReLU': nn.ReLU(),
'Tanh': nn.Tanh(),
'LeakyReLU': nn.LeakyReLU(),
'Sigmoid': nn.Sigmoid()
}


optuna.logging.set_verbosity(optuna.logging.ERROR)
def champion_callback(study, frozen_trial):
    winner = study.user_attrs.get('winner', None)
    if winner is None:
        print(f'Initial trial {frozen_trial.number} achived value: {frozen_trial.value}')
    elif winner != study.best_value and study.best_value:   # second condition is for preventing zero devision
        improvment_percent = (abs(winner - study.best_value) / abs(study.best_value)) * 100
        print(f'Trial {frozen_trial.number} achived value: {frozen_trial.value} with {improvment_percent:.4f}% improvment')
    study.set_user_attr('winner', study.best_value)


def objective(trial):
    params = {
    'batch_size': trial.suggest_categorical('batch_size', [2**i for i in range(4,10)]),
    'device': device,
    'epochs' : epochs,
    # 'optimizer': optimizer.defaults,
    'loss_function': loss_func.__class__.__name__,
    'learning_rate': trial.suggest_float('lr', 1e-4, 1, log=True),
    'num_conv': trial.suggest_int("num_conv", 2, 10),
    'num_pool': trial.suggest_int("num_pool", 0, 2),
    'activation': trial.suggest_categorical("activation", list(activation_functions.keys())),
    'fc_hidden_layer': trial.suggest_int('fc_hidden_layer', 0, 5),
    'init_hidden_size':  trial.suggest_categorical('init_hidden_size', [2**i for i in range(6,10)]),
    }
    model = build_model(num_conv_layers=params['num_conv'],
                        num_pool_layers=params['num_pool'],
                        num_hidden_layer=params['fc_hidden_layer'], 
                        init_hidden_size=params['init_hidden_size'], 
                        activation=activation_functions[params['activation']],
                    )
    
    # run_name = f'trial_lr_{params["learning_rate"]:.8f}'
    run_name = f'trial: {trial.number}'
    with mlflow.start_run(nested=True, run_name=run_name):

        optimizer = torch.optim.SGD(model.parameters(), lr=params['learning_rate'])
        params['optimizer'] = optimizer.defaults
        mlflow.log_params(params)

        train_dataloader = DataLoader(cifar_train, batch_size=params['batch_size'],shuffle=True)
        valid_dataloader = DataLoader(cifar_test, batch_size=params['batch_size'])
        
        with open ("model_summary.txt", "w") as f:
            f.write(str(summary(model)))
        mlflow.log_artifact("model_summary.txt")

        # model.apply(weight_reset)
        cnn_results = train_network(
            model=model,
            loss_func=loss_func,
            train_loader=train_dataloader,
            valid_loader=valid_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            score_funcs={'Acc':accuracy_score_wrapper},
            device=device,
            # checkpont_file_save='model.pth'
        )
        
        # input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        # output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        # signature = ModelSignature(inputs=input_schema, outputs=output_schema)
        # mlflow.pytorch.log_model(model, "model", signature=signature)
        # mlflow.log_figure(plot_results(fc_results), "valid_AUC.png")
    return  cnn_results['valid Acc'].iloc[-1]

epochs = 20
# run_name = "second_attempt"
# with mlflow.start_run():
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20, callbacks=[champion_callback])
champion_trial = study.best_trial
print(f"Champion trial: {champion_trial.number} with value {champion_trial.value}")

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

training:   0%|          | 0/782 [00:00<?, ?it/s]

validating:   0%|          | 0/157 [00:00<?, ?it/s]

training:   0%|          | 0/782 [00:00<?, ?it/s]

validating:   0%|          | 0/157 [00:00<?, ?it/s]

...

training:   0%|          | 0/391 [00:00<?, ?it/s]

validating:   0%|          | 0/79 [00:00<?, ?it/s]

training:   0%|          | 0/391 [00:00<?, ?it/s]

validating:   0%|          | 0/79 [00:00<?, ?it/s]

Champion trial: 4 with value 0.6853


In [ ]:
!zip -r mlflow_exercise_2.zip ./mlruns

## 4

In [5]:
mlflow.set_experiment('Exercise_4')

2025/04/14 11:50:49 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_4' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/399752393838767952', creation_time=1744631449026, experiment_id='399752393838767952', last_update_time=1744631449026, lifecycle_stage='active', name='Exercise_4', tags={}>

In [6]:
# C = cifar_train[0][0].shape[0]
C = 3
filter = 16
K = 3
# w = cifar_train[0][0].shape[1]
# h = cifar_train[0][0].shape[2]
w , h = 32, 32
# classes = cifar_train[0][1].shape[0]
classes = 10
def build_model(num_conv_layers,
                num_pool_layers,
                num_hidden_layer=2, 
                init_hidden_size=512, 
                decay_factor=2,
                activation=nn.ReLU()):
    layers =[]
    in_channels = C
    out_channels = 32
    if num_pool_layers:
        pool_interval = max(1, num_conv_layers // (num_pool_layers + 1))
    else:
        pool_interval = num_conv_layers + 1
    
    currnet_pool_rounds = 0
    for i in range(num_conv_layers):
        layers.append(nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels,
            kernel_size=3,
            padding=3//2))
        layers.append(activation)
        in_channels = out_channels
        if (i+1) % pool_interval == 0 and currnet_pool_rounds < num_pool_layers:
            layers.append(nn.MaxPool2d(kernel_size=2))
            currnet_pool_rounds += 1
            out_channels *= 2
    final_w = w // (2 ** num_pool_layers)
    final_h = h // (2 ** num_pool_layers)
    fc_layers = []
    # Compute the number of features after flattening.
    in_features = in_channels * final_w * final_h

    fc_layers.append(nn.Flatten())

    if num_hidden_layer == 0:
        # Directly classify without extra hidden layers.
        fc_layers.append(nn.Linear(in_features, classes))
    else:
        # First FC layer: from flattened output to initial hidden size.
        fc_layers.append(nn.Linear(in_features, init_hidden_size))
        fc_layers.append(nn.ReLU(inplace=True))
        
        # Set the current hidden size that will be reduced in subsequent layers.
        current_hidden_size = init_hidden_size

        # Add additional hidden layers with decreasing size.
        for layer in range(1, num_hidden_layer):
            # Compute new hidden size with decay.
            new_hidden_size = max(10, current_hidden_size // decay_factor)
            fc_layers.append(nn.Linear(current_hidden_size, new_hidden_size))
            fc_layers.append(nn.ReLU(inplace=True))
            current_hidden_size = new_hidden_size

        # Final classification layer from the last hidden dimension to the number of classes.
        fc_layers.append(nn.Linear(current_hidden_size, classes))

    classifier = nn.Sequential(*fc_layers)
    model = nn.Sequential(*layers, classifier)
    return model

In [7]:
batch_size = 256

In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
cifar_train = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transform,
    train=True,)
cifar_test = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transform,
    train=False,)

cifar_train_loader = DataLoader(
    cifar_train,
    shuffle=True,
    batch_size=batch_size,
    )

cifar_test_loader = DataLoader(
    cifar_test,
    batch_size=batch_size,
    )

100%|██████████| 170M/170M [00:16<00:00, 10.2MB/s] 


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


In [9]:
transforms_augmentation = {
    'RandomAffine1': transforms.RandomAffine(degrees=10),
    'RandomAffine2': transforms.RandomAffine(degrees=0, translate=(0.1,0.1)),
    'RandomAffine3': transforms.RandomAffine(degrees=0, shear=15),
    'RandomHorizontalFlip': transforms.RandomHorizontalFlip(p=0.5),
    'RandomVerticalFlip': transforms.RandomVerticalFlip(p=0.2),
    'RandomPerspective': transforms.RandomPerspective(p=.2),   
    'ColorJitter': transforms.ColorJitter(brightness=0.5, contrast=0.5),
    }

In [15]:
cifar_augmented_train_loader = {}
for trfm_name, trfm in transforms_augmentation.items():
    train_transforms = transforms.Compose(
    [trfm,
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]    
    )
    cifar_augmented_train = torchvision.datasets.CIFAR10(
    f'./data_augmented_cifar_{trfm_name}',
    download=True,
    transform=train_transforms,
    train=True,)

    cifar_augmented_train_loader[trfm_name] = DataLoader(
        cifar_augmented_train,
        shuffle=True,
        batch_size=batch_size,)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 10
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

In [16]:

# model = model_cnn_pool(D=32*32, C=3, filters=128, K=3, classes=10)
train_loaders = {'without_aug': cifar_train_loader,
                 **cifar_augmented_train_loader}

In [17]:
for aug_name, train_loader in train_loaders.items():
    print(aug_name)
    model = build_model(num_conv_layers=4,
            num_pool_layers=2,
            num_hidden_layer=2, 
            init_hidden_size=512, 
            decay_factor=2,
            activation=nn.ReLU())
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    params['optimizer'] = optimizer.defaults
    
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))
    with mlflow.start_run(nested=True, run_name=aug_name):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=cifar_test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

without_aug


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

...

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
!zip -r mlflow_exercise_4.zip ./mlruns


## 5

In [6]:
mlflow.set_experiment('Exercise_5')

2025/04/18 15:07:16 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_5' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/360360039686392042', creation_time=1744988836419, experiment_id='360360039686392042', last_update_time=1744988836419, lifecycle_stage='active', name='Exercise_5', tags={}>

In [7]:
batch_size = 256

In [8]:
mnist_data_train = torchvision.datasets.MNIST(
    "./data",
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    )
mnist_data_test = torchvision.datasets.MNIST(
    "./data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
    )

mnist_train_loader = DataLoader(
    mnist_data_train,
    batch_size=batch_size,
    shuffle=True
    )
mnist_test_loader = DataLoader(
    mnist_data_test,
    batch_size=batch_size
    )

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 49.5MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.88MB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 14.9MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 5.71MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 10
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

### mnist

In [10]:
for i in (2 ** j for j in range(2,10)):
    print(i)
    model = model_cnn_pool(D=28*28, C=1, filters=i, K=3, classes=10)
    params['filters'] = i
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    params['optimizer'] = optimizer.defaults
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))  
    with mlflow.start_run(nested=True, run_name='cnn_pool_mnist'+f'filter={i}'):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=mnist_train_loader,# mnist
            valid_loader=mnist_test_loader,#mnist
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

4


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

training:   0%|          | 0/235 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/235 [00:00<?, ?it/s]

...

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/235 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/235 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

### cifar

In [11]:
cifar_train = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transforms.ToTensor(),
    train=True,)
cifar_test = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transforms.ToTensor(),
    train=False,)

cifar_train_loader = DataLoader(
    cifar_train,
    shuffle=True,
    batch_size=batch_size,
    num_workers=4)

cifar_test_loader = DataLoader(
    cifar_test,
    batch_size=batch_size,
    num_workers=4)

100%|██████████| 170M/170M [00:02<00:00, 80.0MB/s] 


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


In [12]:
# C = cifar_train[0][0].shape[0]
C = 3
filter = 16
K = 3
# w = cifar_train[0][0].shape[1]
# h = cifar_train[0][0].shape[2]
w , h = 32, 32
# classes = cifar_train[0][1].shape[0]
classes = 10
def build_model(num_conv_layers,
                num_pool_layers,
                num_hidden_layer=2, 
                init_hidden_size=512, 
                decay_factor=2,
                activation=nn.ReLU(),
                out_channels=32):
    layers =[]
    in_channels = C
    # out_channels = 32
    if num_pool_layers:
        pool_interval = max(1, num_conv_layers // (num_pool_layers + 1))
    else:
        pool_interval = num_conv_layers + 1
    
    currnet_pool_rounds = 0
    for i in range(num_conv_layers):
        layers.append(nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels,
            kernel_size=3,
            padding=3//2))
        layers.append(activation)
        in_channels = out_channels
        if (i+1) % pool_interval == 0 and currnet_pool_rounds < num_pool_layers:
            layers.append(nn.MaxPool2d(kernel_size=2))
            currnet_pool_rounds += 1
            out_channels *= 2
    final_w = w // (2 ** num_pool_layers)
    final_h = h // (2 ** num_pool_layers)
    fc_layers = []
    # Compute the number of features after flattening.
    in_features = in_channels * final_w * final_h

    fc_layers.append(nn.Flatten())

    if num_hidden_layer == 0:
        # Directly classify without extra hidden layers.
        fc_layers.append(nn.Linear(in_features, classes))
    else:
        # First FC layer: from flattened output to initial hidden size.
        fc_layers.append(nn.Linear(in_features, init_hidden_size))
        fc_layers.append(nn.ReLU(inplace=True))
        
        # Set the current hidden size that will be reduced in subsequent layers.
        current_hidden_size = init_hidden_size

        # Add additional hidden layers with decreasing size.
        for layer in range(1, num_hidden_layer):
            # Compute new hidden size with decay.
            new_hidden_size = max(10, current_hidden_size // decay_factor)
            fc_layers.append(nn.Linear(current_hidden_size, new_hidden_size))
            fc_layers.append(nn.ReLU(inplace=True))
            current_hidden_size = new_hidden_size

        # Final classification layer from the last hidden dimension to the number of classes.
        fc_layers.append(nn.Linear(current_hidden_size, classes))

    classifier = nn.Sequential(*fc_layers)
    model = nn.Sequential(*layers, classifier)
    return model

In [ ]:
for i in (2 ** j for j in range(2,10)):
    print(i)
    params['filters'] = i
    model = build_model(num_conv_layers=4,
            num_pool_layers=2,
            num_hidden_layer=2, 
            init_hidden_size=512, 
            decay_factor=2,
            activation=nn.ReLU(),
            out_channels=i)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    params['optimizer'] = optimizer.defaults
    with open('model_summary.txt', 'w') as f:
        f.write(str(summary(model)))  
    with mlflow.start_run(nested=True, run_name='build_model_cifar'+f'filter={i}'):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')

        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=cifar_train_loader,# mnist
            valid_loader=cifar_test_loader,#mnist
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

4


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

...

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

In [ ]:
!zip -r mlflow_exercise_5_1.zip ./mlruns

## 6

In [5]:
mlflow.set_experiment('Exercise_6')

2025/04/19 05:18:45 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_6' does not exist. Creating a new experiment.


<Experiment: artifact_location='/kaggle/working/mlruns/806400083569447856', creation_time=1745039925875, experiment_id='806400083569447856', last_update_time=1745039925875, lifecycle_stage='active', name='Exercise_6', tags={}>

In [6]:
batch_size = 256

In [7]:

C, H, W = 3, 32, 32
num_pixels = H * W

permutation = torch.randperm(num_pixels)

def fixed_shuffle(img):
    """
    Expects a tensor image of shape [C, H, W]
    """
    # Flatten the spatial dimensions: for each channel, create a (H*W) vector.
    # img.view(C, -1) will give a [C, H*W] tensor.
    img_reshaped = img.view(C, -1)
    
    # Apply the fixed permutation to each channel
    img_shuffled = img_reshaped[:, permutation]
    
    # Reshape back to [C, H, W]
    return img_shuffled.view(C, H, W)

# Now create a transform pipeline using Lambda
shuffle_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(fixed_shuffle),
])


In [8]:
cifar_train_shuffletransform = torchvision.datasets.CIFAR10(
    './data_cifar_shuffletransform',
    download=True,
    transform=shuffle_transform,
    # target_transform=lambda x: torch.tensor(x),
    train=True,)
cifar_test_shuffletransform = torchvision.datasets.CIFAR10(
    './data_cifar_shuffletransform',
    download=True,
    transform=shuffle_transform,
    # target_transform=lambda x: torch.tensor(x),
    train=False,)

cifar_train_loader_shuffle = DataLoader(
    cifar_train_shuffletransform,
    shuffle=True,
    batch_size=batch_size,
    )

cifar_test_loader_shuffle = DataLoader(
    cifar_test_shuffletransform,
    batch_size=batch_size,
    )

100%|██████████| 170M/170M [00:04<00:00, 34.5MB/s] 


Extracting ./data_cifar_shuffletransform/cifar-10-python.tar.gz to ./data_cifar_shuffletransform
Files already downloaded and verified


In [9]:
cifar_train = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transforms.ToTensor(),
    train=True,)
cifar_test = torchvision.datasets.CIFAR10(
    './data_cifar',
    download=True,
    transform=transforms.ToTensor(),
    train=False,)

cifar_train_loader = DataLoader(
    cifar_train,
    shuffle=True,
    batch_size=batch_size,
    )

cifar_test_loader = DataLoader(
    cifar_test,
    batch_size=batch_size,
    )

100%|██████████| 170M/170M [00:04<00:00, 35.3MB/s] 


Extracting ./data_cifar/cifar-10-python.tar.gz to ./data_cifar
Files already downloaded and verified


In [10]:
train_loaders = (cifar_train_loader, cifar_train_loader_shuffle)
test_loaders = (cifar_test_loader, cifar_test_loader_shuffle)
name_experiment = (
    'build_model_cifar',
    'build_model_cifar_shuffletransform',
)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_func = nn.CrossEntropyLoss()
score_funcs = {"Accuracy": accuracy_score_wrapper}
epochs = 20
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size
}

In [12]:
# C = cifar_train[0][0].shape[0]
C = 3
filter = 16
K = 3
# w = cifar_train[0][0].shape[1]
# h = cifar_train[0][0].shape[2]
w , h = 32, 32
# classes = cifar_train[0][1].shape[0]
classes = 10
def build_model(num_conv_layers,
                num_pool_layers,
                num_hidden_layer=2, 
                init_hidden_size=512, 
                decay_factor=2,
                activation=nn.ReLU(),
                out_channels=32):
    layers =[]
    in_channels = C
    # out_channels = 32
    if num_pool_layers:
        pool_interval = max(1, num_conv_layers // (num_pool_layers + 1))
    else:
        pool_interval = num_conv_layers + 1
    
    currnet_pool_rounds = 0
    for i in range(num_conv_layers):
        layers.append(nn.Conv2d(
            in_channels=in_channels, 
            out_channels=out_channels,
            kernel_size=3,
            padding=3//2))
        layers.append(activation)
        in_channels = out_channels
        if (i+1) % pool_interval == 0 and currnet_pool_rounds < num_pool_layers:
            layers.append(nn.MaxPool2d(kernel_size=2))
            currnet_pool_rounds += 1
            out_channels *= 2
    final_w = w // (2 ** num_pool_layers)
    final_h = h // (2 ** num_pool_layers)
    fc_layers = []
    # Compute the number of features after flattening.
    in_features = in_channels * final_w * final_h

    fc_layers.append(nn.Flatten())

    if num_hidden_layer == 0:
        # Directly classify without extra hidden layers.
        fc_layers.append(nn.Linear(in_features, classes))
    else:
        # First FC layer: from flattened output to initial hidden size.
        fc_layers.append(nn.Linear(in_features, init_hidden_size))
        fc_layers.append(nn.ReLU(inplace=True))
        
        # Set the current hidden size that will be reduced in subsequent layers.
        current_hidden_size = init_hidden_size

        # Add additional hidden layers with decreasing size.
        for layer in range(1, num_hidden_layer):
            # Compute new hidden size with decay.
            new_hidden_size = max(10, current_hidden_size // decay_factor)
            fc_layers.append(nn.Linear(current_hidden_size, new_hidden_size))
            fc_layers.append(nn.ReLU(inplace=True))
            current_hidden_size = new_hidden_size

        # Final classification layer from the last hidden dimension to the number of classes.
        fc_layers.append(nn.Linear(current_hidden_size, classes))

    classifier = nn.Sequential(*fc_layers)
    model = nn.Sequential(*layers, classifier)
    return model

In [13]:
model = build_model(num_conv_layers=4,
        num_pool_layers=2,
        num_hidden_layer=2, 
        init_hidden_size=512, 
        decay_factor=2,
        activation=nn.ReLU(),
        out_channels=256)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
params['optimizer'] = optimizer.defaults
with open('model_summary.txt', 'w') as f:
    f.write(str(summary(model)))
for i in range(2):
    print(i)
    train_loader = train_loaders[i]
    test_loader = test_loaders[i]    
    with mlflow.start_run(nested=True, run_name=name_experiment[i]):
        mlflow.log_params(params)
        mlflow.log_artifact('model_summary.txt')
        model.apply(weight_reset)
        results = train_network(
            model=model,
            optimizer=optimizer,
            loss_func=loss_func,
            train_loader=train_loader,
            valid_loader=test_loader,
            epochs=epochs,
            device=device,
            score_funcs=score_funcs,
            )

0


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

...

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

training:   0%|          | 0/196 [00:00<?, ?it/s]

validating:   0%|          | 0/40 [00:00<?, ?it/s]

In [14]:
!zip -r mlflow_exercise_6.zip ./mlruns

  adding: mlruns/ (stored 0%)
  adding: mlruns/806400083569447856/ (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/ (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/tags/ (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/tags/mlflow.source.name (deflated 5%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/tags/mlflow.user (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/tags/mlflow.source.type (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/tags/mlflow.runName (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/artifacts/ (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/artifacts/model_summary.txt (deflated 77%)
  adding: mlruns/806400083569447856/27c75abecaa6490c9a2c145db51acd66/params/ (stored 0%)
  adding: mlruns/806400083569447856/27c75abecaa6